In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
import torch
import os, json, re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging

from huggingface_hub import login
from config import hf_token
login(token=hf_token) # LUCAS STORES HIS HF TOKEN IN CONFIG.PY NOT UPLOADED TO GITHUB (SINCE REPO IS PUBLIC)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/lucasbandarkar/.cache/huggingface/token
Login successful


In [1]:
from wordle import Wordle
from prompting import Prompt_Helper
game = Wordle()
p = Prompt_Helper()

In [5]:
llama3_8b = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(llama3_8b, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(llama3_8b, 
                                             torch_dtype="auto",
                                             device_map="auto", ).to("cuda")
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.58s/it]


In [3]:
conversation = p.get_few_shot_instructions(2)
conversation

[{'role': 'system',
  'content': ('Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position an invalid letter or a correct letter present in the word but in a wrong position. After each guess, we get a result',)},
 {'role': 'system', 'content': 'Example #1:'},
 {'role': 'user', 'content': 'Enter guess #0'},
 {'role': 'assistant', 'content': 'BAGEL'},
 {'role': 'user',
  'content': 'Letter B is not anywhere present in the word, Letter A is not anywhere present in the word, Letter G is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct position, Letter L is not anywhere present in the word, '},
 {'role': 'user', 'content': 'Enter guess #1'},
 {'role': 'assistant', 'content': 'COVER'},
 {'role': 'user',
  'content': 'Letter C is not anywhere present in the word, Letter O is not anywhere present in the word, Letter V is not anywhere present in the word, Letter E at Position 3 is in the word a

In [19]:
# Not Fully tested yet
i = 0
while True:
    input_ids = tokenizer.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=5,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    guess_str = tokenizer.decode(response, skip_special_tokens=True)

    guess_eval = game.turn(guess_str)
    user_response = p.explanatory_response(guess_eval)
    conversation.extend([
        {'role': 'assistant', 'content': guess_str},
        {'role': 'user', 'content': user_response}
    ])
    if guess_eval == 'GGGGG':
        break
    elif i >= 4:
        conversation.append({'role': 'system', 'content': "5 guesses exhausted"})
        break
    else:
        i += 1

conversation

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


COVER
